# Realistic Erosion Model Simulation

This notebook demonstrates a realistic erosion model that:
- Generates random terrain using fractal surfaces
- Creates geological layers with different erodibilities
- Simulates weather and rainfall over time
- Erodes terrain based on water flow and material properties
- Creates rivers and lakes from accumulated water

## Setup

In [ ]:
# Import the erosion model
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.patches import Patch

from erosion_model import (
    ErosionSimulation,
    generate_terrain,
    generate_layer_thicknesses,
    compute_layer_interfaces,
    fill_depressions,
    compute_flow_direction_d8,
    compute_flow_accumulation,
    identify_rivers,
    identify_lakes,
    LAYER_ORDER,
    LAYER_PROPERTIES,
)

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

print("Erosion model loaded successfully!")

## Configuration

Set simulation parameters here:

In [ ]:
# Simulation Parameters
GRID_SIZE = 128           # Grid size (NxN) - increase for more detail, but slower
PIXEL_SCALE_M = 100.0     # Size of each grid cell in meters
ELEV_RANGE_M = 500.0      # Total elevation range in meters
TOTAL_DEPTH_M = 200.0     # Total depth of geological layers

# Erosion parameters
NUM_YEARS = 100           # Number of years to simulate
K_BASE = 1e-4             # Stream power erosion coefficient (higher = faster erosion)
D_BASE = 5e-4             # Hillslope diffusion coefficient (higher = more smoothing)

# Random seed (set to None for random results, or an integer for reproducibility)
RANDOM_SEED = 42

print(f"Grid size: {GRID_SIZE}x{GRID_SIZE} ({GRID_SIZE*GRID_SIZE:,} cells)")
print(f"Domain size: {GRID_SIZE*PIXEL_SCALE_M/1000:.1f} x {GRID_SIZE*PIXEL_SCALE_M/1000:.1f} km")
print(f"Simulation: {NUM_YEARS} years")

## Initialize Simulation

Create the terrain, geological layers, and initialize the erosion simulation:

In [ ]:
# Create the simulation
print("Initializing simulation...")
sim = ErosionSimulation(
    grid_size=GRID_SIZE,
    pixel_scale_m=PIXEL_SCALE_M,
    elev_range_m=ELEV_RANGE_M,
    total_depth_m=TOTAL_DEPTH_M,
    random_seed=RANDOM_SEED,
)

print(f"\nTerrain created:")
print(f"  Elevation range: {sim.elevation.min():.1f} - {sim.elevation.max():.1f} m")
print(f"  Relief: {sim.elevation.max() - sim.elevation.min():.1f} m")

# Count layers
n_layers = sum(1 for t in sim.thicknesses.values() if t.max() > 0.1)
print(f"\nGeological layers created: {n_layers}")
print(f"  Mean surface erodibility: {sim.erodibility.mean():.3f}")

## View Initial State

Plot the initial terrain with rivers and lakes, surface geology, and drainage network:

In [ ]:
# Generate initial water features
print("Computing initial water features...")
filled_elev, lake_depth = fill_depressions(sim.elevation)
flow_dir = compute_flow_direction_d8(filled_elev)
flow_accum = compute_flow_accumulation(filled_elev, flow_dir)
river_mask = identify_rivers(flow_accum, threshold_fraction=0.03)
lake_mask = identify_lakes(lake_depth, min_depth=1.0)

initial_results = {
    "flow_accumulation": flow_accum,
    "river_mask": river_mask,
    "lake_mask": lake_mask,
    "lake_depth": lake_depth,
}

print(f"Initial rivers: {river_mask.sum()} cells")
print(f"Initial lakes: {lake_mask.sum()} cells")

# Plot initial state
sim.plot_state(initial_results, title_prefix="Initial: ")

## View Initial Cross-Section

See the geological layers beneath the surface:

In [ ]:
# Plot cross-section through the middle
sim.plot_cross_section(row=GRID_SIZE // 2)

## Run Erosion Simulation

Now run the multi-year erosion simulation:

In [ ]:
%%time

print(f"Running {NUM_YEARS}-year erosion simulation...")
print("=" * 50)

results = sim.run_simulation(
    num_years=NUM_YEARS,
    K_base=K_BASE,
    D_base=D_BASE,
    save_interval=max(1, NUM_YEARS // 10),  # Save ~10 snapshots
    verbose=True,
)

print("=" * 50)
print("\nSimulation complete!")
print(f"  Final elevation range: {sim.elevation.min():.1f} - {sim.elevation.max():.1f} m")
print(f"  Total erosion: {results['cumulative_erosion'].sum()/1e6:.2f} million m³")
print(f"  Max erosion depth: {results['cumulative_erosion'].max()*1000:.1f} mm")
print(f"  Final rivers: {results['river_mask'].sum()} cells")
print(f"  Final lakes: {results['lake_mask'].sum()} cells")

## View Final State

Plot the terrain after erosion with rivers and lakes:

In [ ]:
# Plot final state
sim.plot_state(results, title_prefix=f"After {NUM_YEARS} years: ")

## View Terrain Evolution

See how the terrain changed over time:

In [ ]:
# Plot evolution over time
sim.plot_evolution()

## View Final Cross-Section

See how the geological layers have been eroded:

In [ ]:
# Plot final cross-section
sim.plot_cross_section(row=GRID_SIZE // 2)

## Compare Before and After

Side-by-side comparison of initial and final terrain:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Initial elevation
ax = axes[0]
initial_elev = sim.history['elevation'][0]
im = ax.imshow(initial_elev, cmap='terrain', origin='lower')
ax.set_title('Initial Elevation')
ax.set_xlabel('X (cells)')
ax.set_ylabel('Y (cells)')
plt.colorbar(im, ax=ax, label='Elevation (m)', shrink=0.8)

# Final elevation
ax = axes[1]
im = ax.imshow(sim.elevation, cmap='terrain', origin='lower')
ax.set_title(f'Final Elevation (after {NUM_YEARS} years)')
ax.set_xlabel('X (cells)')
ax.set_ylabel('Y (cells)')
plt.colorbar(im, ax=ax, label='Elevation (m)', shrink=0.8)

# Elevation change
ax = axes[2]
elev_change = initial_elev - sim.elevation
vmax = max(abs(elev_change.min()), abs(elev_change.max()))
im = ax.imshow(elev_change, cmap='RdBu_r', origin='lower', vmin=-vmax, vmax=vmax)
ax.set_title('Elevation Change (erosion = red)')
ax.set_xlabel('X (cells)')
ax.set_ylabel('Y (cells)')
plt.colorbar(im, ax=ax, label='Change (m)', shrink=0.8)

plt.tight_layout()
plt.show()

print(f"Total elevation lowered: {(initial_elev - sim.elevation).mean()*1000:.2f} mm on average")

## Custom Visualization: Topography with Rivers Highlighted

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

# Base terrain
im = ax.imshow(sim.elevation, cmap='terrain', origin='lower', alpha=1.0)

# Overlay rivers with blue color
river_display = np.ma.masked_where(~results['river_mask'], np.ones_like(sim.elevation))
ax.imshow(river_display, cmap='Blues', origin='lower', alpha=0.9, vmin=0, vmax=1)

# Overlay lakes with darker blue
lake_display = np.ma.masked_where(~results['lake_mask'], results['lake_depth'])
ax.imshow(lake_display, cmap='Blues_r', origin='lower', alpha=0.9)

# Add contour lines
contour_levels = np.linspace(sim.elevation.min(), sim.elevation.max(), 20)
ax.contour(sim.elevation, levels=contour_levels, colors='black', alpha=0.3, linewidths=0.5, origin='lower')

ax.set_title(f'Terrain with Rivers and Lakes after {NUM_YEARS} years', fontsize=14)
ax.set_xlabel('X (grid cells)')
ax.set_ylabel('Y (grid cells)')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Elevation (m)')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='blue', alpha=0.7, label=f'Rivers ({results["river_mask"].sum()} cells)'),
    Patch(facecolor='darkblue', alpha=0.7, label=f'Lakes ({results["lake_mask"].sum()} cells)'),
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

## Analyze Erosion by Layer Type

See which geological layers were most affected by erosion:

In [ ]:
# Analyze erosion patterns
print("Erosion Analysis")
print("=" * 50)

# Get layer statistics
layer_stats = []
for layer in LAYER_ORDER:
    if layer in sim.thicknesses:
        thickness = sim.thicknesses[layer]
        if thickness.max() > 0.1:
            erodibility = LAYER_PROPERTIES[layer]['erodibility']
            layer_stats.append({
                'layer': layer,
                'mean_thickness': thickness.mean(),
                'max_thickness': thickness.max(),
                'erodibility': erodibility,
            })
            print(f"{layer:15s}: thickness {thickness.mean():6.2f}m (max {thickness.max():6.2f}m), erodibility {erodibility:.2f}")

print("\nErosion Statistics:")
print(f"  Mean erosion: {results['cumulative_erosion'].mean()*1000:.2f} mm")
print(f"  Max erosion: {results['cumulative_erosion'].max()*1000:.2f} mm")
print(f"  Erosion std: {results['cumulative_erosion'].std()*1000:.2f} mm")

## Experiment: Change Parameters

Try running a new simulation with different parameters:

In [ ]:
# Create a new simulation with higher erosion rate
print("Running simulation with HIGHER erosion rate...")

sim_fast = ErosionSimulation(
    grid_size=64,  # Smaller for speed
    pixel_scale_m=100.0,
    elev_range_m=400.0,
    total_depth_m=150.0,
    random_seed=123,
)

results_fast = sim_fast.run_simulation(
    num_years=200,
    K_base=5e-4,  # 5x higher erosion
    D_base=1e-3,
    save_interval=20,
    verbose=True,
)

sim_fast.plot_state(results_fast, title_prefix="High Erosion: ")

## Save Results

Save the simulation results for later analysis:

In [ ]:
# Save final terrain to file
np.save('final_elevation.npy', sim.elevation)
np.save('cumulative_erosion.npy', results['cumulative_erosion'])
np.save('river_mask.npy', results['river_mask'])
np.save('lake_mask.npy', results['lake_mask'])

print("Results saved to:")
print("  - final_elevation.npy")
print("  - cumulative_erosion.npy")
print("  - river_mask.npy")
print("  - lake_mask.npy")

---

## Summary

This erosion model demonstrates:

1. **Terrain Generation**: Fractal surfaces with domain warping create realistic topography
2. **Geological Layers**: Multiple layers with different erodibilities (topsoil erodes fast, granite erodes slowly)
3. **Weather Simulation**: Storms with variable intensity and direction produce spatially-varying rainfall
4. **Water Flow**: D8 flow routing accumulates water downslope
5. **Erosion Physics**: Stream power erosion (rivers) + hillslope diffusion create realistic landforms
6. **Lakes**: Depressions fill with water creating lakes
7. **Rivers**: High flow accumulation areas form rivers

Try adjusting the parameters to see different results!